In [1]:

%pprint
%matplotlib inline
import sys
sys.path.insert(1, '../py')
from jobpostlib import (crf, cu, datetime, duration, hau, hc, humanize, ihu, lru, nu, osp, scrfcu, slrcu, ssgdcu, su, t0, time, wsu, speech_engine)
import os
from pandas import DataFrame
import re

freq = 990

Pretty printing has been turned OFF
======== Neo4j/4.4.7 ========
Utility libraries created in 14 seconds


In [7]:

cypher_str = """
    // Get the file names of HTML strings that are Job Titles
    CALL {
        MATCH (np1:NavigableParents)-[r:NEXT]->(np2:NavigableParents)
        WHERE
            ((np1.is_header = false) AND (np1.is_job_title = true))
        RETURN
            r.file_name AS file_name,
            LEFT(np2.navigable_parent, 50) AS left_navigable_parent,
            LEFT(np1.navigable_parent, 50) AS middle_navigable_parent,
            NULL AS right_navigable_parent
    UNION
        MATCH (np2:NavigableParents)-[r:NEXT]->(np1:NavigableParents)
        WHERE
            ((np1.is_header = false) AND (np1.is_job_title = true))
        RETURN
            r.file_name AS file_name,
            LEFT(np2.navigable_parent, 50) AS left_navigable_parent,
            LEFT(np1.navigable_parent, 50) AS middle_navigable_parent,
            NULL AS right_navigable_parent
    }
    RETURN file_name, left_navigable_parent, middle_navigable_parent, right_navigable_parent;"""
row_objs_list = []
with cu.driver.session() as session:
        row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)
if row_objs_list:
    df = DataFrame(row_objs_list)
    df['job_title'] = df.file_name.map(lambda file_name: su.get_job_title_from_file_name(file_name))
    display(df)

,file_name,left_navigable_parent,middle_navigable_parent,right_navigable_parent,job_title
0,3c2a56e15051e119_Training_and_placement_with_B...,2.,<b>Python Developer</b>,None,Training and placement with BA QA remote Remote
1,77201e16ca4e59f7_Python_Developer_Los_Angeles_...,<div>\n 4+ years of proven experience as a Py...,<b>Python Developer</b>,None,Python Developer Los Angeles CA
2,a750c4fced248190_Python_Developer_Maharashtra_...,- Maharashtra,<b>Python Developer</b>,None,Python Developer Maharashtra Los Angeles CA
3,8daead9543820e35_Senior_Software_Engineer_Pyth...,<b>Job Description:</b>,<b>Python Developer</b>,None,Senior Software Engineer Python Remote
4,65eff1bfff618e30_Sr_Python_Developer_Remote_In...,<b>Job Description:</b>,<b>Python Developer</b>,None,Sr Python Developer Remote
...,...,...,...,...,...
630,cf771117afc5276e_Assistant_Research_Professor_...,The Department of Biomedical Informatics at th...,"<i>Assistant Research Professor Position, Onto...",None,Assistant Research Professor in and Curation A...
631,5a11f57c84f46ede_Senior_Machine_Learning_Engin...,<p>#LI-REMOTE</p>,<p> #LI-MidSenior</p>,None,Senior Machine Learning Engineer Machine Learn...
632,6b2b2944837c5137_Advanced_Data_Analyst_Remote_...,Job Title:,<b>Advanced Data Analyst </b>,None,Advanced Data Analyst Remote
633,9901043347759693_Biostatistician_Remote_Indeed...,Regular,<div>\n Clearance Level Must Currently Posse...,None,9901043347759693 Remote


In [6]:

# mask_series = (df.char_count > 165)
for child_str in df.iloc[637:642].middle_navigable_parent:
    def do_cypher_tx(tx, navigable_parent):
        cypher_str = """
    // Set the is_job_title to false
    MATCH (np:NavigableParents {navigable_parent: $navigable_parent})
    SET np.is_job_title = false
    """ + cu.return_everything_str + ';'
        print(cypher_str.replace('$navigable_parent', f'"{child_str}"'))
        return [dict(record.items()) for record in tx.run(query=cypher_str, parameters={'navigable_parent': navigable_parent})]
    with cu.driver.session() as session:
        row_objs_list = session.write_transaction(do_cypher_tx, navigable_parent=child_str)
    # display(DataFrame(row_objs_list).T)
    # break


    // Set the is_job_title to false
    MATCH (np:NavigableParents {navigable_parent: "<div>
   Clearance Level Must Currently Possess:
 "})
    SET np.is_job_title = false
    RETURN
        np.navigable_parent AS navigable_parent,
        np.is_header AS is_header,
        np.is_task_scope AS is_task_scope,
        np.is_minimum_qualification AS is_minimum_qualification,
        np.is_preferred_qualification AS is_preferred_qualification,
        np.is_legal_notification AS is_legal_notification,
        np.is_job_title AS is_job_title,
        np.is_office_location AS is_office_location,
        np.is_job_duration AS is_job_duration,
        np.is_supplemental_pay AS is_supplemental_pay,
        np.is_educational_requirement AS is_educational_requirement,
        np.is_interview_procedure AS is_interview_procedure,
        np.is_corporate_scope AS is_corporate_scope,
        np.is_posting_date AS is_posting_date,
        np.is_other AS is_other;

    // Set the is_job_title to fal

In [4]:

cypher_str = """
// Get all distinct node types and property names
CALL db.schema.nodeTypeProperties() YIELD nodeType, propertyName
RETURN DISTINCT nodeType, propertyName;"""
row_objs_list = []
with cu.driver.session() as session: row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)
if row_objs_list:
    df = DataFrame(row_objs_list)
    df.nodeType = df.nodeType.map(lambda x: re.sub('[^A-Za-z]+', '', str(x)))
    display(df)

,nodeType,propertyName
0,HeaderTags,is_in_document_structure_elements_set
1,HeaderTags,is_in_document_head_elements_set
2,HeaderTags,is_in_document_body_elements_set
3,HeaderTags,is_in_block_elements_set
4,HeaderTags,is_in_other_block_elements_set
...,...,...
94,PartsOfSpeech,is_corporate_scope
95,PartsOfSpeech,is_posting_date
96,PartsOfSpeech,is_other
97,PartsOfSpeech,pos_symbol


In [5]:

mask_series = df.propertyName.map(lambda x: 'job' in str(x))
df[mask_series]

,nodeType,propertyName
43,FileNames,job_requisition_number
69,NavigableParents,is_job_title
71,NavigableParents,is_job_duration
88,PartsOfSpeech,is_job_title
90,PartsOfSpeech,is_job_duration


In [6]:

cypher_str = """
// Get all node types that have the is_job_title property defined
CALL db.schema.nodeTypeProperties() YIELD nodeType, propertyName
WHERE propertyName = "is_job_title"
RETURN DISTINCT nodeType;"""
row_objs_list = []
with cu.driver.session() as session: row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)
if row_objs_list:
    df = DataFrame(row_objs_list)
    display(df)

,nodeType
0,:`NavigableParents`
1,:`PartsOfSpeech`


In [108]:

cypher_str = """
    // Get all the HTML strings that are considered job titles
    MATCH (np:NavigableParents)
    WHERE
        (np.is_job_title = true)
        AND (np.is_header = false)
    RETURN
        np.is_corporate_scope AS is_corporate_scope,
        np.is_educational_requirement AS is_educational_requirement,
        np.is_header AS is_header,
        np.is_interview_procedure AS is_interview_procedure,
        np.is_job_duration AS is_job_duration,
        np.is_job_title AS is_job_title,
        np.is_legal_notification AS is_legal_notification,
        np.is_minimum_qualification AS is_minimum_qualification,
        np.is_office_location AS is_office_location,
        np.is_other AS is_other,
        np.is_posting_date AS is_posting_date,
        np.is_preferred_qualification AS is_preferred_qualification,
        np.is_supplemental_pay AS is_supplemental_pay,
        np.is_task_scope AS is_task_scope,
        np.navigable_parent AS navigable_parent
    ORDER BY size(np.navigable_parent) ASC;"""
row_objs_list = []
with cu.driver.session() as session:
        row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)
if row_objs_list:
    df = DataFrame(row_objs_list)
    df['char_count'] = df.navigable_parent.map(lambda x: len(str(x)))
    columns_list = ['navigable_parent', 'is_header', 'char_count']
    display(df[columns_list].sort_values('char_count'))

,navigable_parent,is_header,char_count
0,Senior Consultant,False,17
1,Backend Developers,False,18
2,Title:Data Scientist,False,20
3,<b>SQL Developer</b>,False,20
4,Python Web Developer,False,20
...,...,...,...
6608,"<span style=""font-family: 'Verdana',sans-serif...",False,97
6609,<strong>Data Scientist / Machine Learning Scie...,False,100
6610,Security Data Science Consultant with Artifici...,False,103
6611,"<h2 class=""jobsearch-JobDescriptionSection-tit...",False,117


In [105]:

# mask_series = (df.char_count > 165)
for child_str in df.iloc[9939:9940].navigable_parent:
    def do_cypher_tx(tx, navigable_parent):
        cypher_str = """
    // Set the is_job_title to false
    MATCH (np:NavigableParents {navigable_parent: $navigable_parent})
    SET np.is_job_title = false
    """ + cu.return_everything_str + ';'
        print(cypher_str.replace('$navigable_parent', f'"{child_str}"'))
        return [dict(record.items()) for record in tx.run(query=cypher_str, parameters={'navigable_parent': navigable_parent})]
    with cu.driver.session() as session:
        row_objs_list = session.write_transaction(do_cypher_tx, navigable_parent=child_str)
    # display(DataFrame(row_objs_list).T)
    # break


    // Set the is_job_title to false
    MATCH (np:NavigableParents {navigable_parent: "positions within the Census Bureau in the same geographical location with the same qualifications and specialized experience."})
    SET np.is_job_title = false
    RETURN
        np.navigable_parent AS navigable_parent,
        np.is_header AS is_header,
        np.is_task_scope AS is_task_scope,
        np.is_minimum_qualification AS is_minimum_qualification,
        np.is_preferred_qualification AS is_preferred_qualification,
        np.is_legal_notification AS is_legal_notification,
        np.is_job_title AS is_job_title,
        np.is_office_location AS is_office_location,
        np.is_job_duration AS is_job_duration,
        np.is_supplemental_pay AS is_supplemental_pay,
        np.is_educational_requirement AS is_educational_requirement,
        np.is_interview_procedure AS is_interview_procedure,
        np.is_corporate_scope AS is_corporate_scope,
        np.is_posting_date AS is_posting

In [109]:

cypher_str = """
    // Set the is_job_title to false
    MATCH (np:NavigableParents)
    WHERE np.navigable_parent CONTAINS "Sr. NLP Researcher"
    SET np.is_job_title = true
    RETURN
        np.navigable_parent AS navigable_parent,
        np.is_header AS is_header,
        np.is_task_scope AS is_task_scope,
        np.is_minimum_qualification AS is_minimum_qualification,
        np.is_preferred_qualification AS is_preferred_qualification,
        np.is_legal_notification AS is_legal_notification,
        np.is_job_title AS is_job_title,
        np.is_office_location AS is_office_location,
        np.is_job_duration AS is_job_duration,
        np.is_supplemental_pay AS is_supplemental_pay,
        np.is_educational_requirement AS is_educational_requirement,
        np.is_interview_procedure AS is_interview_procedure,
        np.is_corporate_scope AS is_corporate_scope,
        np.is_posting_date AS is_posting_date,
        np.is_other AS is_other;"""
row_objs_list = []
with cu.driver.session() as session: row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)
if row_objs_list:
    df = DataFrame(row_objs_list)
    display(df.T)

,0
navigable_parent,"<h2 style=""font-family:'Helvetica Neue',Helvet..."
is_header,False
is_task_scope,False
is_minimum_qualification,False
is_preferred_qualification,False
is_legal_notification,False
is_job_title,True
is_office_location,False
is_job_duration,False
is_supplemental_pay,False


In [112]:

cypher_str = """
// Get all node types that have a file property defined
CALL db.schema.nodeTypeProperties() YIELD nodeType, propertyName
WHERE propertyName CONTAINS "file"
RETURN DISTINCT nodeType, propertyName;"""
row_objs_list = []
with cu.driver.session() as session:
        row_objs_list = session.write_transaction(cu.do_cypher_tx, cypher_str)
if row_objs_list:
    df = DataFrame(row_objs_list)
    display(df)

,nodeType,propertyName
0,,file_name
1,,file_name_id
2,:`FileNames`,file_name
3,:`FileNames`,file_name_id
